### 1. 变长序列的处理方式理解

变长序列的处理逻辑就是：

👉 虽然原始的变长序列（如 genres、keywords）在输入时是一个序列，但在经过 embedding 和 pooling（这里是 mean pooling）之后，会被压缩成一个定长向量，相当于一个“整体语义”表示。
换句话说：

原始序列（如 ['动作', '冒险', '科幻']）
⬇️
通过 Embedding → 变成 (batch_size, seq_len, embedding_dim) 的张量
⬇️
通过 mean pooling → 汇总成 (batch_size, 1, embedding_dim)
⬇️
和其它稀疏特征一起参与拼接、建模
所以最终：

无论是稀疏特征（单个索引）还是变长特征（序列），最终都统一成 (batch_size, 1, embedding_dim) 的形式，这样才能拼接起来喂给 FM 和 DNN。

这种做法其实就是“把变长特征转化为定长特征”的通用套路之一（还有比如 attention pooling、CNN、RNN 都可以做类似事情）。

### 2. 整体程序

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

# 1. 加载并预处理数据，这里模拟出2个变长序列数据
data = pd.read_csv(r"D:\software\pycharm_repository\StarMaker\MultiRecSys\data_files\movielens_sample.txt")
data['genres_bak'] = data['genres']
data.head()

sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ["rating"]
sequence_features = ["genres", "genres_bak"]

# 对稀疏特征做标签编码（Label Encoding）
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

# 用于存储每个变长特征处理后的 padding 序列
pad_sequences_dict = {}

# 每个变长特征对应一个独立的 Tokenizer，用于后续文本转索引
tokenizers = {}

# 用于记录每个变长特征的 padding 长度（即序列被填充后的最大长度）
pad_len_dict = {}

# 遍历所有变长序列特征
for feature in sequence_features:
    texts = data[feature].apply(lambda x: x.replace('|', ' ')).tolist()
    tokenizer = Tokenizer(oov_token='OOV')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, padding='post')  # shape: (num_samples, max_seq_len)
    pad_sequences_dict[feature] = padded
    tokenizers[feature] = tokenizer
    pad_len_dict[feature] = padded.shape[1]  # 保存每个特征的序列长度（max_seq_len）

# 2. 创建所有特征的Embedding层
embedding_dim = 4
vocab_sizes = {feat: data[feat].nunique() for feat in sparse_features}

for feature in sequence_features:
    feat_num = len(tokenizers[feature].word_index) + 1
    vocab_sizes[feature] = feat_num

# 创建嵌入层字典
embed_layers = {}
for feat in sparse_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=False)
for feat in sequence_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=True)

# 1. 初始化模型输入字典
inputs = {}
for feat in sparse_features:
    inputs[feat] = tf.keras.Input(shape=(1,), name=feat, dtype=tf.int32)  # shape: (batch_size, 1)
for feat in sequence_features:
    max_len = pad_len_dict[feat]
    inputs[feat] = tf.keras.Input(shape=(max_len,), name=feat, dtype=tf.int32)  # shape: (batch_size, max_len)

# 2. 构建嵌入列表
embeds = []
for feat in sparse_features:
    embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, 1, embedding_dim)
    embeds.append(embed)

for feat in sequence_features:
    seq_embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, seq_len, embedding_dim)
    pooled_embed = tf.reduce_mean(seq_embed, axis=1, keepdims=True)  # shape: (batch_size, 1, embedding_dim) 从这可以看出边长序列的字段数据最终整体也是当成一个字段处理
    embeds.append(pooled_embed)

# 拼接所有嵌入特征
total_embeds = tf.concat(embeds, axis=1)  # shape: (batch_size, num_fields, embedding_dim)

# 4. FM 二阶交叉项计算
sum_square = tf.square(tf.reduce_sum(total_embeds, axis=1))  # shape: (batch_size, embedding_dim)
square_sum = tf.reduce_sum(tf.square(total_embeds), axis=1)  # shape: (batch_size, embedding_dim)
fm_second_order = 0.5 * tf.reduce_sum(sum_square - square_sum, axis=1, keepdims=True)  # shape: (batch_size, 1)

# 5. DNN 部分
flatten_input = tf.reshape(total_embeds, shape=(-1, total_embeds.shape[1] * embedding_dim))  # shape: (batch_size, num_fields * embedding_dim)
x = layers.Dense(64, activation='relu')(flatten_input)  # shape: (batch_size, 64)
x = layers.Dense(32, activation='relu')(x)              # shape: (batch_size, 32)
dnn_output = layers.Dense(1)(x)                         # shape: (batch_size, 1)

# 6. 合并 FM 和 DNN 输出结果
output = layers.Add()([fm_second_order, dnn_output])    # shape: (batch_size, 1)
model = tf.keras.Model(inputs=list(inputs.values()), outputs=output)

# 7. 编译并训练模型
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# 构建模型输入字典
model_input = {}
for feat in sparse_features:
    model_input[feat] = data[feat].values  # shape: (num_samples,)
for feat in sequence_features:
    model_input[feat] = pad_sequences_dict[feat]  # shape: (num_samples, max_seq_len)

# 模型训练
model.fit(model_input, data[target].values, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 14.1148 - mse: 14.1148 - val_loss: 13.2294 - val_mse: 13.2294
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 14.0121 - mse: 14.0121 - val_loss: 13.1344 - val_mse: 13.1344
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 13.9067 - mse: 13.9067 - val_loss: 13.0361 - val_mse: 13.0361
Epoch 4/10
1/1 [==============================] - 0s 26ms/step - loss: 13.7975 - mse: 13.7975 - val_loss: 12.9349 - val_mse: 12.9349
Epoch 5/10
1/1 [==============================] - 0s 27ms/step - loss: 13.6829 - mse: 13.6829 - val_loss: 12.8296 - val_mse: 12.8296
Epoch 6/10
1/1 [==============================] - 0s 26ms/step - loss: 13.5639 - mse: 13.5639 - val_loss: 12.7197 - val_mse: 12.7197
Epoch 7/10
1/1 [==============================] - 0s 43ms/step - loss: 13.4389 - mse: 13.4389 - val_loss: 12.6029 - val_mse: 12.6029
Epoch 8/10
1/1 [==============================] - 0s 26ms/step - loss: 

### 3.详细注释

In [41]:
# 1. 加载并预处理数据，这里模拟出2个变长序列数据
data = pd.read_csv(r"D:\software\pycharm_repository\StarMaker\MultiRecSys\data_files\movielens_sample.txt")
data['genres_bak'] = data['genres']
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip,genres_bak
0,3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119,Comedy|Drama
1,3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005,Action|Thriller
2,517,105,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,F,25,14,55408,Drama|Romance
3,785,2115,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,18,19,29307,Action|Adventure
4,5848,909,5,957782527,"Apartment, The (1960)",Comedy|Drama,M,50,20,20009,Comedy|Drama


In [42]:
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ["rating"]
sequence_features = ["genres", "genres_bak"]

In [43]:
# 对稀疏特征做标签编码（Label Encoding）
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [44]:
# 用于存储每个变长特征处理后的 padding 序列
pad_sequences_dict = {}

# 每个变长特征对应一个独立的 Tokenizer，用于后续文本转索引
tokenizers = {}

# 用于记录每个变长特征的 padding 长度（即序列被填充后的最大长度）
pad_len_dict = {}

In [45]:
# 遍历所有变长序列特征
for feature in sequence_features:
    texts = data[feature].apply(lambda x: x.replace('|', ' ')).tolist()
    tokenizer = Tokenizer(oov_token='OOV')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, padding='post')  # shape: (num_samples, max_seq_len)
    pad_sequences_dict[feature] = padded
    tokenizers[feature] = tokenizer
    pad_len_dict[feature] = padded.shape[1]  # 保存每个特征的序列长度（max_seq_len）

In [46]:
pad_sequences_dict

{'genres': array([[2, 3, 0, 0, 0, 0],
        [4, 5, 0, 0, 0, 0],
        [3, 6, 0, 0, 0, 0],
        ...,
        [2, 6, 0, 0, 0, 0],
        [4, 9, 5, 0, 0, 0],
        [2, 0, 0, 0, 0, 0]]),
 'genres_bak': array([[2, 3, 0, 0, 0, 0],
        [4, 5, 0, 0, 0, 0],
        [3, 6, 0, 0, 0, 0],
        ...,
        [2, 6, 0, 0, 0, 0],
        [4, 9, 5, 0, 0, 0],
        [2, 0, 0, 0, 0, 0]])}

In [47]:
tokenizers

{'genres': <keras_preprocessing.text.Tokenizer at 0x2d80c922a60>,
 'genres_bak': <keras_preprocessing.text.Tokenizer at 0x2d7dcc67ac0>}

In [48]:
pad_len_dict

{'genres': 6, 'genres_bak': 6}

In [49]:
# 2. 创建所有特征的Embedding层
embedding_dim = 4
vocab_sizes = {feat: data[feat].nunique() for feat in sparse_features}

for feature in sequence_features:
    feat_num = len(tokenizers[feature].word_index) + 1
    vocab_sizes[feature] = feat_num

In [50]:
vocab_sizes

{'movie_id': 187,
 'user_id': 193,
 'gender': 2,
 'age': 7,
 'occupation': 20,
 'zip': 188,
 'genres': 21,
 'genres_bak': 21}

In [51]:
# 创建嵌入层字典
embed_layers = {}
for feat in sparse_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=False)
for feat in sequence_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=True)

In [52]:
embed_layers

{'movie_id': <keras.layers.embeddings.Embedding at 0x2d80c922610>,
 'user_id': <keras.layers.embeddings.Embedding at 0x2d80c922670>,
 'gender': <keras.layers.embeddings.Embedding at 0x2d80c922700>,
 'age': <keras.layers.embeddings.Embedding at 0x2d80c922f40>,
 'occupation': <keras.layers.embeddings.Embedding at 0x2d80c9f2160>,
 'zip': <keras.layers.embeddings.Embedding at 0x2d80c9f20d0>,
 'genres': <keras.layers.embeddings.Embedding at 0x2d80c922c40>,
 'genres_bak': <keras.layers.embeddings.Embedding at 0x2d80c9f28e0>}

In [53]:
# 1. 初始化模型输入字典
inputs = {}
for feat in sparse_features:
    inputs[feat] = tf.keras.Input(shape=(1,), name=feat, dtype=tf.int32)  # shape: (batch_size, 1)
for feat in sequence_features:
    max_len = pad_len_dict[feat]
    inputs[feat] = tf.keras.Input(shape=(max_len,), name=feat, dtype=tf.int32)  # shape: (batch_size, max_len)
inputs

{'movie_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 'user_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 'gender': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 'age': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 'zip': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 'genres': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 'genres_bak': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>}

In [54]:
# 2. 构建嵌入列表
embeds = []
for feat in sparse_features:
    embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, 1, embedding_dim)
    embeds.append(embed)

for feat in sequence_features:
    seq_embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, seq_len, embedding_dim)
    pooled_embed = tf.reduce_mean(seq_embed, axis=1, keepdims=True)  # shape: (batch_size, 1, embedding_dim) 从这可以看出边长序列的字段数据最终整体也是当成一个字段处理
    embeds.append(pooled_embed)

In [55]:
embeds

[<KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_24')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_25')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_26')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_27')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_28')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_29')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'tf.math.reduce_mean_6')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'tf.math.reduce_mean_7')>]

In [56]:
# 拼接所有嵌入特征
total_embeds = tf.concat(embeds, axis=1)  # shape: (batch_size, num_fields, embedding_dim)
total_embeds

<KerasTensor: shape=(None, 8, 4) dtype=float32 (created by layer 'tf.concat_4')>

In [57]:
# 4. FM 二阶交叉项计算
sum_square = tf.square(tf.reduce_sum(total_embeds, axis=1))  # shape: (batch_size, embedding_dim)
square_sum = tf.reduce_sum(tf.square(total_embeds), axis=1)  # shape: (batch_size, embedding_dim)
fm_second_order = 0.5 * tf.reduce_sum(sum_square - square_sum, axis=1, keepdims=True)  # shape: (batch_size, 1)

In [58]:
# 5. DNN 部分
flatten_input = tf.reshape(total_embeds, shape=(-1, total_embeds.shape[1] * embedding_dim))  # shape: (batch_size, num_fields * embedding_dim)
x = layers.Dense(64, activation='relu')(flatten_input)  # shape: (batch_size, 64)
x = layers.Dense(32, activation='relu')(x)              # shape: (batch_size, 32)
dnn_output = layers.Dense(1)(x)                         # shape: (batch_size, 1)

In [59]:
# 6. 合并 FM 和 DNN 输出结果
output = layers.Add()([fm_second_order, dnn_output])    # shape: (batch_size, 1)
output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_4')>

In [60]:
model = tf.keras.Model(inputs=list(inputs.values()), outputs=output)

In [61]:
model

In [62]:
inputs

{'movie_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 'user_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 'gender': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 'age': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 'zip': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 'genres': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 'genres_bak': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>}

In [63]:
inputs.values()

dict_values([<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>, <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>, <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>])

In [64]:
inputs=list(inputs.values())
inputs

[<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>]

In [65]:
# 7. 编译并训练模型
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# 构建模型输入字典
model_input = {}
for feat in sparse_features:
    model_input[feat] = data[feat].values  # shape: (num_samples,)
for feat in sequence_features:
    model_input[feat] = pad_sequences_dict[feat]  # shape: (num_samples, max_seq_le

In [66]:
model_input

{'movie_id': array([ 12, 169,   6, 112,  45, 146,  43, 156,  30, 174,  82, 173,  91,
        108, 132,  40, 109,  31, 180, 183, 129,  67, 137,  87, 127,   8,
        104, 100, 140,  25, 122, 124, 116, 126,  72, 117,  42, 145, 131,
          2,  52,  17, 101,  94, 136,  65,  20, 144,  26,  83,  55, 126,
        184,  23, 121, 142,  33,   0,  46, 139, 150, 135,  36, 110,  79,
        162,  70, 147,   9,  34,   7,  76,   4, 185,  73, 112, 130,  95,
         28,  24, 148, 119, 168, 149, 181,  13, 154,  56,  66, 172,  69,
         35,  49, 106,  35,  11, 152, 166,  37, 164,  54, 167,  72,  29,
         92, 114,  88, 170,  64,  60,  38,  22,  62, 178, 134, 157,  99,
         34, 111,  96,  50,  75,  47,  14,  21,  77, 118, 182, 113, 143,
        149, 141,  10,  58,  81,  44,  27, 151, 165,  98, 163,  80, 158,
        161,  27, 155, 171,  78,  57, 123,  84,  93, 170, 120,   1, 153,
         39,  61,  51,  71,  19, 107,   9,  66, 102,  74, 177, 103, 133,
        160,  53,  90,   5, 173,  41,  

In [67]:
# 模型训练
model.fit(model_input, data[target].values, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 1s 943ms/step - loss: 14.2872 - mse: 14.2872 - val_loss: 13.4153 - val_mse: 13.4153
Epoch 2/10
1/1 [==============================] - 0s 26ms/step - loss: 14.1986 - mse: 14.1986 - val_loss: 13.3399 - val_mse: 13.3399
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 14.1121 - mse: 14.1121 - val_loss: 13.2637 - val_mse: 13.2637
Epoch 4/10
1/1 [==============================] - 0s 46ms/step - loss: 14.0272 - mse: 14.0272 - val_loss: 13.1860 - val_mse: 13.1860
Epoch 5/10
1/1 [==============================] - 0s 46ms/step - loss: 13.9424 - mse: 13.9424 - val_loss: 13.1077 - val_mse: 13.1077
Epoch 6/10
1/1 [==============================] - 0s 25ms/step - loss: 13.8566 - mse: 13.8566 - val_loss: 13.0273 - val_mse: 13.0273
Epoch 7/10
1/1 [==============================] - 0s 25ms/step - loss: 13.7692 - mse: 13.7692 - val_loss: 12.9449 - val_mse: 12.9449
Epoch 8/10
1/1 [==============================] - 0s 26ms/step - los